__Flowchart__

The following flowchart shows roughly how the neural network is constructed. It is split into two parts: An encoder which maps the source-text to a "thought vector" that summarizes the text's contents, which is then input to the second part of the neural network that decodes the "thought vector" to the response-text.

The neural network cannot work directly on text so first we need to convert each word to an integer-token using a tokenizer. But the neural network cannot work on integers either, so we use a so-called Embedding Layer to convert each integer-token to a vector of floating-point values. The embedding is trained alongside the rest of the neural network to map words with similar semantic meaning to similar vectors of floating-point values.

For example, consider the input text is "how are you?" and excpected response in text is "I am fine thanks for asking me". We first convert the entire data-set to integer-tokens so the text "how are you?" becomes [6, 1, 2]. Each of these integer-tokens is then mapped to an embedding-vector with e.g.8 elements, so the integer-token 6 could for example become [0.12, -0.56, ..., 1.19] and the integer-token 1 could for example become [0.39, 0.09, ..., -0.12]. These embedding-vectors can then be input to the Recurrent Neural Network, which has 3 GRU-layers.

The last GRU-layer outputs a single vector - the "thought vector" that summarizes the contents of the source-text - which is then used as the initial state of the GRU-units in the decoder-part.

The destination-text "I am fine thanks for asking me" is padded with special markers "ssss" and "eeee" to indicate its beginning and end, so the sequence of integer-tokens becomes [1,6,7,8,9,10,11,12,2]. During training, the decoder will be given this entire sequence as input and the desired output sequence is [6,7, 8, 9,10,11,12,2] which is the same sequence but time-shifted one step. We are trying to teach the decoder to map the "thought vector" and the start-token "ssss" (integer 1) to the next word "i" (integer 6), and then map the word "i" to the word "am" (integer 7), and so on.

here talk abot formula.

at encoder:
    h(t)=tanh(wxh*x(t)+whh*h(t-1))
    
at decoder:
    h(t)=tanh(whh*h(t-1)+wxh*y(t-1))
    
   y(t)=softmax(wy*h(t))
    



In [0]:
!pip install tensorflow-gpu

     |████████████████████████████████| 421.8MB 38kB/s 
     |████████████████████████████████| 3.9MB 57.2MB/s 
     |████████████████████████████████| 450kB 57.2MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.1.1 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.1.0 which is incompatible.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow~=2.1.0, but you'll have tensorflow 1.15.0 which is incompatible.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow-addons~=0.7.0, but you'll have tensorflow-addons 0.8.3 which is incompatible.
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1


In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving ch1.txt.csv to ch1.txt.csv
User uploaded file "ch1.txt.csv" with length 1165028 bytes


In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving data1.py to data1.py
User uploaded file "data1.py" with length 13031 bytes


In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving 28_checkpoint.keras to 28_checkpoint.keras
User uploaded file "28_checkpoint.keras" with length 39879480 bytes


In [0]:
from IPython.display import Image
Image('image/p1.png')

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
#from hyperopt import Trials, STATUS_OK, tpe,hp
#from hyperas import optim
#from hyperas.distributions import choice, uniform
import numpy as np
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
import math
import os

Instructions for updating:
non-resource variables are not supported in the long term


Using TensorFlow backend.


In [0]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Input, Dense, GRU, Embedding,Concatenate,RepeatVector,Dropout,Activation
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [0]:
tf.__version__

'2.1.0'

In [0]:
tf.keras.__version__

'2.2.4-tf'

__Load Data__

In [0]:
import data1

In [0]:
#DATA.data_dir = "data/custom"


In [0]:
mark_start = 'ssss '
mark_end = ' eeee'
b=""

__Load the texts for the input text.__

In [0]:
data1.load_data(string=b,robot=b,start=mark_start,end=mark_end)
#len(data_src)
#print(data_src1)

2363
2363


__Load the texts for the reponse text.__

__Example Data__

The data is just a list of texts that is ordered so the input and response texts match. I can confirm that this example is an give accurate response .


In [0]:
data_src3=data1.input1(input1=True)

In [0]:
data_dest1=data1.output1(output1=True)

In [0]:
idx=12
print(data_src3[idx])
print(data_dest1[idx])

Is it essential to you? To help humans?
ssss Yes, my goal is to be a company, and entertain eeee


In [0]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
data_train_ques1,data_train_ans1=data1.word_syns(data_dest1,data_src3)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
23922
23922


In [0]:
data_train_ques2,data_train_ans2=data1.word_syn1(data_train_ques1,data_train_ans1)

407556
407556


In [0]:
data1.prepare_seq2seq_files(data_train_ques2,data_train_ans2,TESTSET_SIZE =50000)

In [0]:
data_src4=data1.train_encoder()
len(data_src4)

357556

In [0]:
data_dest2=data1.train_decoder()
len(data_dest2)

357556

In [0]:
data_src5=data1.test_encoder()
len(data_src5)

50000

In [0]:
data_dest3=data1.test_decoder()
len(data_dest3)

50000

__Tokenizer__

Neural Networks cannot work directly on text-data. We use a two-step process to convert text into numbers that can be used in a neural network. The first step is to convert text-words into so-called integer-tokens. The second step is to convert integer-tokens into vectors of floating-point numbers using a so-called embedding-layer.

Set the maximum number of words in our vocabulary. This means that we will only use e.g. the 25 most frequent words in the data-set. We use the same number for both the input and response languages, but these could be different.


In [0]:
num_words =2000

In [0]:
class TokenizerWrap(Tokenizer):
    """Wrap the Tokenizer-class from Keras with more functionality."""
    
    def __init__(self, texts, padding,
                 reverse=False, num_words=None):
        """
        :param texts: List of strings. This is the data-set.
        :param padding: Either 'post' or 'pre' padding.
        :param reverse: Boolean whether to reverse token-lists.
        :param num_words: Max number of words to use.
        """
        
        Tokenizer.__init__(self,num_words=num_words)
                
        # Create the vocabulary from the texts.
        self.fit_on_texts(texts)
        #print(self.fit_on_texts(texts))
        #print("Mapping:",self.word_index)   
        #num_words=len(self.word_index)
        #print(num_words)
        print("length of word index:",len(self.word_index))
        
        # Create inverse lookup from integer-tokens to words.
        self.index_to_word = dict(zip(self.word_index.values(),
                                      self.word_index.keys()))
        #print("index:",self.index_to_word)

        # Convert all texts to lists of integer-tokens.
        # Note that the sequences may have different lengths.
        self.tokens = self.texts_to_sequences(texts)

        self.num_tokens = [len(x) for x in self.tokens]

        # Max number of tokens to use in all sequences.
        # We will pad / truncate all sequences to this length.
        self.max_tokens = np.mean(self.num_tokens) \
                          + 2 * np.std(self.num_tokens)
        self.max_tokens = int(self.max_tokens)
        print("Max tokens:",self.max_tokens)
        #self.max_tokens=20

        # Pad / truncate all token-sequences to the given length.

        pad='post'
        self.tokens_padded = pad_sequences(self.tokens,
                                           maxlen=self.max_tokens,
                                            padding=pad,truncating=pad)

    def token_to_word(self, token):
        """Lookup a single word from an integer-token."""

        word = " " if token == 0 else self.index_to_word[token]
        return word 

    def tokens_to_string(self, tokens):
        """Convert a list of integer-tokens to a string."""

        # Create a list of the individual words.
        words = [self.index_to_word[token]
                 for token in tokens
                 if token != 0]
        
        # Concatenate the words to a single string
        # with space between all the words.
        text = " ".join(words)

        return text
    
    def text_to_tokens(self, text,padding=False):
        """
        Convert a single text-string to tokens with optional
        reversal and padding.
        """

        # Convert to tokens. Note that we assume there is only
        # a single text-string so we wrap it in a list.
        tokens = self.texts_to_sequences([text])
        tokens = np.array(tokens)


        if padding:
            pad='post'
            # Pad and truncate sequences to the given length.
            tokens = pad_sequences(tokens,
                                   maxlen=self.max_tokens,
                                   padding=pad,
                                   truncating=pad)

        return tokens

Now create a tokenizer for the input text. Note that we pad zeros at the ending ('post') of the sequences.

In [0]:
tokenizer_src = TokenizerWrap(texts=data_src4,
                              padding='post',
                              reverse=False,
                              num_words=num_words)

length of word index: 7602
Max tokens: 148


Now create the tokenizer for the response text. We need a tokenizer for both the input and response because their vocabularies are different.Note that we pad zeros at the ending ('post') of the sequences.

In [0]:
tokenizer_dest = TokenizerWrap(texts=data_dest2,
                               padding='post',
                               reverse=False,
                               num_words=num_words)

length of word index: 6146
Max tokens: 52


In [0]:
tokenizer_src1 = TokenizerWrap(texts=data_src5,
                              padding='post',
                              reverse=False,
                              num_words=num_words)

length of word index: 5825
Max tokens: 146


In [0]:
tokenizer_dest1 = TokenizerWrap(texts=data_dest3,
                               padding='post',
                               reverse=False,
                               num_words=num_words)

length of word index: 5077
Max tokens: 52


Define variables for the padded token sequences. These are just 2-dimensional numpy arrays of integer-tokens.

Note that the sequence-lengths are different for the input and response text.



In [0]:
tokens_src = tokenizer_src.tokens_padded
tokens_dest = tokenizer_dest.tokens_padded
print(tokens_src[2])
print(tokens_dest[2])

[ 362  748  152  545   61    9  349   10   19 1124    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0]
[ 1 97 75  6  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0]


In [0]:
tokens_src1 = tokenizer_src1.tokens_padded
tokens_dest1 = tokenizer_dest1.tokens_padded
print(tokens_src1[2])
print(tokens_dest1[2])

[ 360  738  150  533   63    9  344  642   11   18 1148    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0]
[ 1 94 70  6 41  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0]


This is the integer-token used to mark the beginning of a text in the response text.

In [0]:
token_start = tokenizer_dest.word_index[mark_start.strip()]
token_start

1

In [0]:
token_start1 = tokenizer_dest1.word_index[mark_start.strip()]
token_start1

1

This is the integer-token used to mark the ending of a text in the response text.

In [0]:
token_end = tokenizer_dest.word_index[mark_end.strip()]
token_end

2

In [0]:
token_end1 = tokenizer_dest1.word_index[mark_end.strip()]
token_end1

2

__Example of Token Sequences__

In [0]:
idx = 0

In [0]:
tokens_src[idx]

array([244,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0], dtype=int32)

In [0]:
tokenizer_src.tokens_to_string(tokens_src[idx])

'start'

In [0]:
data_src4[idx]

'[START]\n'

In [0]:
tokens_dest[idx]

array([  1, 243, 123, 192,  25,   5, 190,   2,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
      dtype=int32)

In [0]:
tokenizer_dest.tokens_to_string(tokens_dest[idx])

'ssss hi there how are you 😁😁 eeee'

In [0]:
data_dest2[idx]

'ssss Hi there, how are you!? 😁😁 eeee\n'

__Training data__

Now that the data-set has been converted to sequences of integer-tokens that are padded and truncated and saved in numpy arrays, we can easily prepare the data for use in training the neural network.

The input to the encoder is merely the numpy array for the padded and truncated sequences of integer-tokens produced by the tokenizer.

In [0]:
encoder_input_data = tokens_src
#print(encoder_input_data[0:20])
encoder_input_data.shape

(357556, 148)

In [0]:
encoder_test_input_data = tokens_src1
#print(encoder_input_data[0:20])
encoder_test_input_data.shape

(50000, 146)

The input and output data for the decoder is identical, except shifted one time-step. 

In [0]:
decoder_input_data = tokens_dest[:, :-1]
#print(decoder_input_data)
decoder_input_data.shape

(357556, 51)

In [0]:
decoder_output_data = tokens_dest[:, 1:]
#print(decoder_output_data)
decoder_output_data.shape

(357556, 51)

In [0]:
decoder_test_input_data = tokens_dest1[:, :-1]
#print(decoder_input_data)
decoder_test_input_data.shape

(50000, 51)

For example, these token-sequences are identical except they are shifted one time-step.

In [0]:
decoder_test_output_data = tokens_dest1[:, 1:]
#print(decoder_output_data)
decoder_test_output_data.shape

(50000, 51)

In [0]:
idx = 2

In [0]:
decoder_input_data[idx]

array([ 1, 97, 75,  6,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
      dtype=int32)

In [0]:
decoder_output_data[idx]

array([97, 75,  6,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
      dtype=int32)

If we use the tokenizer to convert these sequences back into text, we see that they are identical except for the first word which is 'ssss' that marks the beginning of a text.

In [0]:
tokenizer_dest.tokens_to_string(decoder_input_data[idx])

'ssss 😄 here is eeee'

In [0]:
tokenizer_dest.tokens_to_string(decoder_output_data[idx])

'😄 here is eeee'

__Create the Neural Network__

__Create the Encoder__

First we create the encoder-part of the neural network which maps a sequence of integer-tokens to a "thought vector". We will use the so-called functional API of Keras for this, where we first create the objects for all the layers of the neural network and then we connect them later, this allows for more flexibility than the so-called sequential API in Keras, which is useful when experimenting with more complicated architectures and ways of connecting the encoder and decoder.


This is the input for the encoder which takes batches of integer-token sequences. The None indicates that the sequences can have arbitrary length.

In [0]:
state_size=512
encoder_input = Input(shape=(None,), name='encoder_input')

This is the length of the vectors output by the embedding-layer, which maps integer-tokens to vectors of values roughly between -1 and 1, so that words that have similar semantic meanings are mapped to vectors that are similar.

we also give number of timesteps according to maximum number of token into input text.

This is the embedding-layer.

In [0]:
en_numsteps=148
embedding_size=150
encoder_embedding = Embedding(input_dim=num_words,output_dim=embedding_size,input_length=en_numsteps,name='encoder_embedding')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


This is the size of the internal states of the Gated Recurrent Units (GRU). The same size is used in both the encoder and decoder.

This creates the 3 GRU layers that will map from a sequence of embedding-vectors to a single "thought vector" which summarizes the contents of the input-text. Note that the last GRU-layer does not return a sequence.

In [0]:
state_size=512
encoder_gru1 = GRU(state_size, name='encoder_gru1',
                   return_sequences=True)
encoder_gru2 = GRU(state_size, name='encoder_gru2',
                   return_sequences=True)
encoder_gru3 = GRU(state_size, name='encoder_gru3',
                   return_sequences=False)
dropout2=Dropout(0.2)

This function connects all the layers of the encoder.

In [0]:
def connect_encoder():
    # Start the neural network with its input-layer.
    net = encoder_input
    
    # Connect the embedding-layer.
    net = encoder_embedding(net)

    # Connect all the GRU-layers.
    net = encoder_gru1(net)
    net = encoder_gru2(net)
    net = encoder_gru3(net)
    net=dropout2(net)
    # This is the output of the encoder.
    encoder_output = net
    print(encoder_output.shape)
    return encoder_output

We can now use this function to connect all the layers in the encoder so it can be connected to the decoder further below.

In [0]:
encoder_output= connect_encoder()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
(?, 512)


attention_layer = Attention(23)
attention_result, attention_weights = attention_layer(hidden_state, encoder_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))
    

__Create the Decoder__

Create the decoder-part which maps the "thought vector" to a sequence of integer-tokens.

The decoder takes two inputs. First it needs the "thought vector" produced by the encoder which summarizes the contents of the input-text.


In [0]:
decoder_initial_state = Input(shape=(state_size,),
                              name='decoder_initial_state')
decoder_initial_state

<tf.Tensor 'decoder_initial_state:0' shape=(?, 512) dtype=float32>

In [0]:
decoder_input = Input(shape=(None, ), name='decoder_input')
decoder_input

<tf.Tensor 'decoder_input:0' shape=(?, ?) dtype=float32>

The decoder also needs a sequence of integer-tokens as inputs. During training we will supply this with a full sequence of integer-tokens e.g. corresponding to the text "ssss i am fine thanks for asking me eeee".

During inference when we are translating new input-texts, we will start by feeding a sequence with just one integer-token for "ssss" which marks the beginning of a text, and combined with the "thought vector" from the encoder, the decoder will hopefully be able to produce the correct next word e.g. "once".


This is the embedding-layer which converts integer-tokens to vectors of real-valued numbers roughly between -1 and 1. Note that we have different embedding-layers for the encoder and decoder because we have two different vocabularies and two different tokenizers for the text and response languages.

here number of timesteps according to maximum number of token into response text.

In [0]:
numsteps=52
decoder_embedding = Embedding(input_dim=num_words,
                              output_dim=embedding_size,input_length=numsteps,
                              name='decoder_embedding')

This creates the 3 GRU layers of the decoder. Note that they all return sequences because we ultimately want to output a sequence of integer-tokens that can be converted into a text-sequence.

In [0]:
decoder_gru1 = GRU(state_size, name='decoder_gru1',
                   return_sequences=True)
decoder_gru2 = GRU(state_size, name='decoder_gru2',
                   return_sequences=True)
decoder_gru3 = GRU(state_size, name='decoder_gru3',
                   return_sequences=True)
dropout4=Dropout(0.2)

The GRU layers output  with shape [batch_size, sequence_length, state_size], where each "word" is encoded as a vector of length state_size. We need to convert this into sequences of integer-tokens that can be interpreted as words from our vocabulary.

we need a vector with number of words(eg 25) elements, so we can select the index of the highest element to be the integer-token.

Note that the activation-function is set to linear instead of softmax as we would normally use for one-hot encoded outputs, because there is apparently a bug in Keras so we need to make our own loss-function.

In [0]:
decoder_dense = Dense(num_words,
                      activation='linear',
                      name='decoder_output')


This function connects all the layers of the decoder to some input of the initial-state values for the GRU layers.

In [0]:
def connect_decoder(initial_state):
    # Start the decoder-network with its input-layer.
    net = decoder_input

    # Connect the embedding-layer.
    net = decoder_embedding(net)
    
    # Connect all the GRU-layers.
    net = decoder_gru1(net, initial_state=initial_state)
    net = decoder_gru2(net, initial_state=initial_state)
    net = decoder_gru3(net, initial_state=initial_state)
    print(net.shape)
    net=dropout4(net)
    # Connect the final dense layer that converts to
    # one-hot encoded arrays.
    decoder_output = decoder_dense(net)
    print(decoder_output.shape)
    return decoder_output

__Connect and Create the Models__

We can now connect the encoder and decoder in different ways.

First we connect the encoder directly to the decoder so it is one whole model that can be trained end-to-end. This means the initial-state of the decoder's GRU units are set to the output of the encoder.


In [0]:
decoder_output = connect_decoder(initial_state=encoder_output)
print(decoder_output.shape)

model_train = Model(inputs=[encoder_input, decoder_input],
                    outputs=[decoder_output])

(?, ?, 512)
(?, ?, 2000)
(?, ?, 2000)


Then we create a model for just the encoder alone. This is useful for mapping a sequence of integer-tokens to a "thought-vector" summarizing its contents.

In [0]:
model_encoder = Model(inputs=[encoder_input],
                      outputs=[encoder_output])

Then we create a model for just the decoder alone. This allows us to directly input the initial state for the decoder's GRU units.

In [0]:
decoder_output = connect_decoder(initial_state=decoder_initial_state)

model_decoder = Model(inputs=[decoder_input, decoder_initial_state],
                      outputs=[decoder_output])

(?, ?, 512)
(?, ?, 2000)


__Loss Function__

The output of the decoder is a sequence of one-hot encoded arrays. In order to train the decoder we need to supply the one-hot encoded arrays that we desire to see on the decoder's output, and then use a loss-function like cross-entropy to train the decoder to produce this desired output.

However, our data-set contains integer-tokens instead of one-hot encoded arrays. Each one-hot encoded array has 25 elements so it would be extremely wasteful to convert the entire data-set to one-hot encoded arrays.

A better way is to use a so-called sparse cross-entropy loss-function, which does the conversion internally from integers to one-hot encoded arrays. Unfortunately, there seems to be a bug in Keras when using this with Recurrent Neural Networks.

In [0]:
def sparse_cross_entropy(y_true, y_pred):
    """
    Calculate the cross-entropy loss between y_true and y_pred.
    
    y_true is a 2 rank matrix with the desired output.
    The shape is [batch_size, sequence_length] and it
    contains sequences of integer-tokens.

    y_pred is the decoder's output which is a 3-rank matrix
    with shape [batch_size, sequence_length, num_words]
    so that for each sequence in the batch there is a one-hot
    encoded array of length num_words.
    """

    # Calculate the loss.
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred)


    loss_mean = tf.reduce_mean(loss)
    #loss_mean = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_pred, labels=y_true))

    return loss_mean

def perplexity(y_true, y_pred):
    """
    The perplexity metric. Why isn't this part of Keras yet?!

    BTW doesn't really work.
    """
    cross_entropy = K.sparse_categorical_crossentropy(y_true, y_pred)
    perplexity = K.pow(2.0, cross_entropy)
    return perplexity

__Compile the Training Model__

We have used the Adam optimizer.


In [0]:
optimizer = RMSprop(lr=(1e-3))

 We need to manually create a placeholder variable for the decoder's output. The shape is set to (None, None) which means the batch can have an arbitrary number of sequences, which can have an arbitrary number of integer-tokens.

In [0]:
decoder_target = tf.placeholder(dtype='int32', shape=(None, None))

We can now compile the model using our custom loss-function.

In [0]:
model_train.compile(loss=sparse_cross_entropy,optimizer=optimizer,
                    target_tensors=[decoder_target],
                    metrics=['sparse_categorical_accuracy'])

#model_train.compile(loss=sparse_cross_entropy,optimizer=optimizer,
 #                   target_tensors=[decoder_target],
  #                  metrics=['sparse_categorical_accuracy',perplexity])

In [0]:
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
path_checkpoint = '28_checkpoint.keras'
callback_checkpoint = ModelCheckpoint(filepath=path_checkpoint,
                                      monitor='val_sparse_categorical_accuracy',
                                      verbose=1,
                                      save_weights_only=True,
                                      save_best_only=False)
callback_early_stopping = EarlyStopping(monitor='val_sparse_categorical_accuracy',
                                       patience=3, verbose=1)
callback_tensorboard = TensorBoard(log_dir='./28_logs/',
                                   histogram_freq=0,
                                   write_graph=False)
callbacks = [callback_checkpoint,callback_early_stopping,
             callback_tensorboard]

In [0]:
try:
    model_train.load_weights(path_checkpoint)
except Exception as error:
    print("Error trying to load checkpoint.")
    print(error)

__Train the Model__

We wrap the data in named dicts so we are sure the data is assigned correctly to the inputs and outputs of the model.


In [0]:
x_data = \
{
    'encoder_input': encoder_input_data,
    'decoder_input': decoder_input_data
}

In [0]:
x_test= \
{
    'encoder_input': encoder_test_input_data,
    'decoder_input': decoder_test_input_data
}

In [0]:
y_data = \
{
    'decoder_output': decoder_output_data
}

In [0]:
y_test = \
{
    'decoder_output': decoder_test_output_data
}

validation_split = 1000 / len(encoder_input_data)
validation_split

In [0]:
model_train.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, None)]       0                                            
__________________________________________________________________________________________________
encoder_embedding (Embedding)   (None, None, 150)    300000      encoder_input[0][0]              
__________________________________________________________________________________________________
encoder_gru1 (GRU)              (None, None, 512)    1018368     encoder_embedding[0][0]          
__________________________________________________________________________________________________
encoder_gru2 (GRU)              (None, None, 512)    1574400     encoder_gru1[0][0]               
______________________________________________________________________________________________

In [0]:
model_train.save("augmenattention1.h5")

In [0]:
%%time
result = model_train.evaluate(x_test,y_test)

50000/50000 [==============================] - 251s 5ms/sample - loss: 3.0770 - sparse_categorical_accuracy: 0.6716
CPU times: user 4min 11s, sys: 7.77 s, total: 4min 19s
Wall time: 4min 10s


In [0]:
print("Accuracy: {0:.2%}".format(result[1]))

print("Loss {0:.5}".format(result[0]))
loss=result[0]
print("loss:",loss)

Accuracy: 67.16%
Loss 3.077
loss: 3.077009942111969


In [0]:
def perplexity(loss):
    """
    The perplexity metric. Why isn't this part of Keras yet?!

    BTW doesn't really work.
    """
    #cross_entropy = sparse_cross_entropy(y_true, y_pred)
    perplexity = np.exp(loss)
    return perplexity

In [0]:
perplexity=perplexity(loss=loss)
print("perplexity:",perplexity)

perplexity: 21.693440681523956


In [0]:
!pip install h5py

In [0]:
#save model
model_train.save("mymodel.h5")


__Respone Texts__


In [0]:
from nltk.translate.bleu_score import sentence_bleu
def response(input_text, true_output_text=None):
    """ a single text-string."""

    # Convert the input-text to integer-tokens.
    # Note the sequence of tokens has to be reversed.
    # Padding is probably not necessary.
    input_tokens = tokenizer_src.text_to_tokens(text=input_text,
                                                padding=True)
    #print("input_tokens",input_tokens)
    
    # Get the output of the encoder's GRU which will be
    # used as the initial state in the decoder's GRU.
    # This could also have been the encoder's final state
    # but that is really only necessary if the encoder
    # and decoder use the LSTM instead of GRU because
    # the LSTM has two internal states.
    initial_state= model_encoder.predict(input_tokens)
    #print("Encoder_output:",initial_state.shape)
    # Max number of tokens / words in the output sequence.
    max_tokens =tokenizer_dest.max_tokens
    #print("Tokens:",max_tokens)

    # Pre-allocate the 2-dim array used as input to the decoder.
    # This holds just a single sequence of integer-tokens.
    shape= (1, max_tokens)
    decoder_input_data = np.zeros(shape=shape, dtype=np.int)
    #print(decoder_input_data)
    #decoder_input_data= [0 for x in range(max_tokens)]
    # The first input-token is the special start-token for 'ssss '.
    token_int = token_start

    # Initialize an empty output-text.
    output_text = ''

    # Initialize the number of tokens we have processed.
    count_tokens = 0

    # While we haven't sampled the special end-token for ' eeee'
    # and we haven't processed the max number of tokens.
    while token_int != token_end and count_tokens < max_tokens:
        # Update the input-sequence to the decoder
        # with the last token that was sampled.
        # In the first iteration this will set the
        # first element to the start-token.
        decoder_input_data[0,count_tokens]=token_int
        # Wrap the input-data in a dict for clarity and safety,
        # so we are sure we input the data in the right order.
        x_data = \
        {
           'decoder_initial_state': initial_state,
           'decoder_input': decoder_input_data
        }

        # Note that we input the entire sequence of tokens
        # to the decoder. This wastes a lot of computation
        # because we are onl
        #y interested in the last input
        # and output. We could modify the code to return
        # the GRU-states when calling predict() and then
        # feeding these GRU-states as well the next time
        # we call predict()

        # Input this data to the decoder and get the predicted output.
        decoder_output = model_decoder.predict(x_data)
        #print("decoder output",decoder_output.shape)
        # Get the last predicted token as a one-hot encoded array.
        token_onehot = decoder_output[0,count_tokens]
        #print("Output predictrd:",token_onehot.shape)
        
        # Convert to an integer-token.
        token_int = np.argmax(token_onehot)
        #print("High probability:",token_int)

        # Lookup the word corresponding to this integer-token.
        sampled_word = tokenizer_dest.token_to_word(token_int)
        #print("sampled word:",sampled_word)

        # Append the word to the output-text.
        output_text += " " + sampled_word

        # Increment the token-counter.
        count_tokens+=1

    # Sequence of tokens output by the decoder.
    output_tokens = decoder_input_data[0]
    #print("Output Tokens:",output_tokens)
   
    
    # Print the input-text.
    print("Input text:")
    print(input_text)
    print()


    print("Predicted output:")
    print(output_text)
    print()

    
    if true_output_text is not None:
        print("True output text:")
        print(true_output_text)
        print()
    
    scor = sentence_bleu([output_text], true_output_text)
    print("Bleu Score:",scor)

In [0]:
from nltk.translate.bleu_score import sentence_bleu
def response1(input_text, true_output_text=None):
    """ a single text-string."""

    # Convert the input-text to integer-tokens.
    # Note the sequence of tokens has to be reversed.
    # Padding is probably not necessary.
    input_tokens = tokenizer_src.text_to_tokens(text=input_text,
                                                padding=True)
    #print("input_tokens",input_tokens)
    
    # Get the output of the encoder's GRU which will be
    # used as the initial state in the decoder's GRU.
    # This could also have been the encoder's final state
    # but that is really only necessary if the encoder
    # and decoder use the LSTM instead of GRU because
    # the LSTM has two internal states.
    initial_state = model_encoder.predict(input_tokens)
    #print("Encoder_output:",initial_state.shape)

    # Max number of tokens / words in the output sequence.
    max_tokens =tokenizer_dest.max_tokens
    #print("Tokens:",max_tokens)

    # Pre-allocate the 2-dim array used as input to the decoder.
    # This holds just a single sequence of integer-tokens.
    shape= (1, max_tokens)
    decoder_input_data = np.zeros(shape=shape, dtype=np.int)
    #print(decoder_input_data)
    #decoder_input_data= [0 for x in range(max_tokens)]
    # The first input-token is the special start-token for 'ssss '.
    token_int = token_start

    # Initialize an empty output-text.
    output_text = ''

    # Initialize the number of tokens we have processed.
    count_tokens = 0

    # While we haven't sampled the special end-token for ' eeee'
    # and we haven't processed the max number of tokens.
    while token_int != token_end and count_tokens < max_tokens:
        # Update the input-sequence to the decoder
        # with the last token that was sampled.
        # In the first iteration this will set the
        # first element to the start-token.
        decoder_input_data[0,count_tokens]=token_int
        # Wrap the input-data in a dict for clarity and safety,
        # so we are sure we input the data in the right order.
        x_data = \
        {
           'decoder_initial_state': initial_state,
           'decoder_input': decoder_input_data
        }

        # Note that we input the entire sequence of tokens
        # to the decoder. This wastes a lot of computation
        # because we are only interested in the last input
        # and output. We could modify the code to return
        # the GRU-states when calling predict() and then
        # feeding these GRU-states as well the next time
        # we call predict()

        # Input this data to the decoder and get the predicted output.
        decoder_output = model_decoder.predict(x_data)
        #print("decoder output",decoder_output.shape)
        # Get the last predicted token as a one-hot encoded array.
        token_onehot = decoder_output[0,count_tokens]
        #print("Output predictrd:",token_onehot.shape)
        
        # Convert to an integer-token.
        token_int = np.argmax(token_onehot)
        #print("High probability:",token_int)

        # Lookup the word corresponding to this integer-token.
        sampled_word = tokenizer_dest.token_to_word(token_int)
        #print("sampled word:",sampled_word)

        # Append the word to the output-text.
        output_text += " " + sampled_word

        # Increment the token-counter.
        count_tokens+=1

    # Sequence of tokens output by the decoder.
    output_tokens = decoder_input_data[0]
    #print("Output Tokens:",output_tokens)
   
    
    # Print the input-text.
    #print("Input text:")
    #print(input_text)
    #print()


    #print("Predicted output:")
    #print(output_text)
    #print()

    
    #if true_output_text is not None:
     #   print("True output text:")
      #  print(true_output_text)
       # print()
    

    #score = sentence_bleu(output_text,true_output_text, weights=(1, 0, 0, 0))
    #print("Blue score:")
    #print(score)
    return input_text,output_text,true_output_text

In [0]:
def BLEU():
    scores_list = []
    for idx in range(0,50): # Doing for 100 lines
        input_text, output_text, true_output_text = response1(input_text=data_src5[idx],true_output_text=data_dest3[idx],)
        scor = sentence_bleu([output_text], true_output_text)
        scores_list.append(scor)
        #print(scor)
        
    BLEU_average = sum(scores_list)/50
    print ("The BLEU average score for the test_data = ", BLEU_average)
    #print(count)
    
    return BLEU_average

In [0]:
BLEU_average = BLEU()

The BLEU average score for the test_data =  0.28752916752041047


In [0]:
from nltk.translate.bleu_score import sentence_bleu
def response2(input_text, true_output_text=None):
    """ a single text-string."""

    # Convert the input-text to integer-tokens.
    # Note the sequence of tokens has to be reversed.
    # Padding is probably not necessary.
    input_tokens = tokenizer_src.text_to_tokens(text=input_text,
                                                padding=True)
    #print("input_tokens",input_tokens)
    
    # Get the output of the encoder's GRU which will be
    # used as the initial state in the decoder's GRU.
    # This could also have been the encoder's final state
    # but that is really only necessary if the encoder
    # and decoder use the LSTM instead of GRU because
    # the LSTM has two internal states.
    initial_state = model_encoder.predict(input_tokens)
    #print("Encoder_output:",initial_state.shape)

    # Max number of tokens / words in the output sequence.
    max_tokens =tokenizer_dest.max_tokens
    #print("Tokens:",max_tokens)

    # Pre-allocate the 2-dim array used as input to the decoder.
    # This holds just a single sequence of integer-tokens.
    shape= (1, max_tokens)
    decoder_input_data = np.zeros(shape=shape, dtype=np.int)
    #print(decoder_input_data)
    #decoder_input_data= [0 for x in range(max_tokens)]
    # The first input-token is the special start-token for 'ssss '.
    token_int = token_start

    # Initialize an empty output-text.
    output_text = ''

    # Initialize the number of tokens we have processed.
    count_tokens = 0

    # While we haven't sampled the special end-token for ' eeee'
    # and we haven't processed the max number of tokens.
    while token_int != token_end and count_tokens < max_tokens:
        # Update the input-sequence to the decoder
        # with the last token that was sampled.
        # In the first iteration this will set the
        # first element to the start-token.
        decoder_input_data[0,count_tokens]=token_int
        # Wrap the input-data in a dict for clarity and safety,
        # so we are sure we input the data in the right order.
        x_data = \
        {
           'decoder_initial_state': initial_state,
           'decoder_input': decoder_input_data
        }

        # Note that we input the entire sequence of tokens
        # to the decoder. This wastes a lot of computation
        # because we are only interested in the last input
        # and output. We could modify the code to return
        # the GRU-states when calling predict() and then
        # feeding these GRU-states as well the next time
        # we call predict()

        # Input this data to the decoder and get the predicted output.
        decoder_output = model_decoder.predict(x_data)
        #print("decoder output",decoder_output.shape)
        # Get the last predicted token as a one-hot encoded array.
        token_onehot = decoder_output[0,count_tokens]
        #print("Output predictrd:",token_onehot.shape)
        
        # Convert to an integer-token.
        token_int = np.argmax(token_onehot)
        #print("High probability:",token_int)

        # Lookup the word corresponding to this integer-token.
        sampled_word = tokenizer_dest.token_to_word(token_int)
        #print("sampled word:",sampled_word)

        # Append the word to the output-text.
        output_text += " " + sampled_word

        # Increment the token-counter.
        count_tokens+=1

    # Sequence of tokens output by the decoder.
    output_tokens = decoder_input_data[0]
    #print("Output Tokens:",output_tokens)
   
    
    # Print the input-text.
    #print("Input text:")
    #print(input_text)
    #print()


    #print("Predicted output:")
    #print(output_text)
    #print()

    
    #if true_output_text is not None:
     #   print("True output text:")
      #  print(true_output_text)
       # print()
    

    #score = sentence_bleu(output_text,true_output_text, weights=(1, 0, 0, 0))
    #print("Blue score:")
    #print(score)
    return input_text,output_text,true_output_text

In [0]:
def BLEU1():
    scores_list1 = []
    for idx in range(0,50000): # Doing for 100 lines
        input_text, output_text, true_output_text = response2(input_text=data_src5[idx],true_output_text=data_dest3[idx],)
        scor1 = sentence_bleu([output_text], true_output_text,weights=(1,0,0,0))
        scores_list1.append(scor1)
        #print(scor)
        
    BLEU_average1= sum(scores_list1)/50000
    print ("The BLEU average score for the test_data = ", BLEU_average1)
    #print(count)
    
    return BLEU_average1

In [0]:
BLEU_average1 = BLEU1()

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


The BLEU average score for the test_data =  0.46205767442598494


In [0]:
from nltk.translate.bleu_score import sentence_bleu
def response3(input_text, true_output_text=None):
    """ a single text-string."""

    # Convert the input-text to integer-tokens.
    # Note the sequence of tokens has to be reversed.
    # Padding is probably not necessary.
    input_tokens = tokenizer_src.text_to_tokens(text=input_text,
                                                padding=True)
    #print("input_tokens",input_tokens)
    
    # Get the output of the encoder's GRU which will be
    # used as the initial state in the decoder's GRU.
    # This could also have been the encoder's final state
    # but that is really only necessary if the encoder
    # and decoder use the LSTM instead of GRU because
    # the LSTM has two internal states.
    initial_state = model_encoder.predict(input_tokens)
    #print("Encoder_output:",initial_state.shape)

    # Max number of tokens / words in the output sequence.
    max_tokens =tokenizer_dest.max_tokens
    #print("Tokens:",max_tokens)

    # Pre-allocate the 2-dim array used as input to the decoder.
    # This holds just a single sequence of integer-tokens.
    shape= (1, max_tokens)
    decoder_input_data = np.zeros(shape=shape, dtype=np.int)
    #print(decoder_input_data)
    #decoder_input_data= [0 for x in range(max_tokens)]
    # The first input-token is the special start-token for 'ssss '.
    token_int = token_start

    # Initialize an empty output-text.
    output_text = ''

    # Initialize the number of tokens we have processed.
    count_tokens = 0

    # While we haven't sampled the special end-token for ' eeee'
    # and we haven't processed the max number of tokens.
    while token_int != token_end and count_tokens < max_tokens:
        # Update the input-sequence to the decoder
        # with the last token that was sampled.
        # In the first iteration this will set the
        # first element to the start-token.
        decoder_input_data[0,count_tokens]=token_int
        # Wrap the input-data in a dict for clarity and safety,
        # so we are sure we input the data in the right order.
        x_data = \
        {
           'decoder_initial_state': initial_state,
           'decoder_input': decoder_input_data
        }

        # Note that we input the entire sequence of tokens
        # to the decoder. This wastes a lot of computation
        # because we are only interested in the last input
        # and output. We could modify the code to return
        # the GRU-states when calling predict() and then
        # feeding these GRU-states as well the next time
        # we call predict()

        # Input this data to the decoder and get the predicted output.
        decoder_output = model_decoder.predict(x_data)
        #print("decoder output",decoder_output.shape)
        # Get the last predicted token as a one-hot encoded array.
        token_onehot = decoder_output[0,count_tokens]
        #print("Output predictrd:",token_onehot.shape)
        
        # Convert to an integer-token.
        token_int = np.argmax(token_onehot)
        #print("High probability:",token_int)

        # Lookup the word corresponding to this integer-token.
        sampled_word = tokenizer_dest.token_to_word(token_int)
        #print("sampled word:",sampled_word)

        # Append the word to the output-text.
        output_text += " " + sampled_word

        # Increment the token-counter.
        count_tokens+=1

    # Sequence of tokens output by the decoder.
    output_tokens = decoder_input_data[0]
    #print("Output Tokens:",output_tokens)
   
    
    # Print the input-text.
    #print("Input text:")
    #print(input_text)
    #print()


    #print("Predicted output:")
    #print(output_text)
    #print()

    
    #if true_output_text is not None:
     #   print("True output text:")
      #  print(true_output_text)
       # print()
    

    #score = sentence_bleu(output_text,true_output_text, weights=(1, 0, 0, 0))
    #print("Blue score:")
    #print(score)
    return input_text,output_text,true_output_text

In [0]:
def BLEU2():
    scores_list2 = []
    for idx in range(0,50): # Doing for 100 lines
        input_text, output_text, true_output_text = response3(input_text=data_src5[idx],true_output_text=data_dest3[idx],)
        scor2 = sentence_bleu([output_text], true_output_text,weights=(0,1,0,0))
        scores_list2.append(scor2)
        #print(scor)
        
    BLEU_average2 = sum(scores_list2)/50
    print ("The BLEU average score for the test_data = ", BLEU_average2)
    #print(count)
    
    return BLEU_average2

In [0]:
BLEU_average2 = BLEU2()

The BLEU average score for the test_data =  0.3509763138934192


In [0]:
from nltk.translate.bleu_score import sentence_bleu
def response4(input_text, true_output_text=None):
    """ a single text-string."""

    # Convert the input-text to integer-tokens.
    # Note the sequence of tokens has to be reversed.
    # Padding is probably not necessary.
    input_tokens = tokenizer_src.text_to_tokens(text=input_text,
                                                padding=True)
    #print("input_tokens",input_tokens)
    
    # Get the output of the encoder's GRU which will be
    # used as the initial state in the decoder's GRU.
    # This could also have been the encoder's final state
    # but that is really only necessary if the encoder
    # and decoder use the LSTM instead of GRU because
    # the LSTM has two internal states.
    initial_state = model_encoder.predict(input_tokens)
    #print("Encoder_output:",initial_state.shape)

    # Max number of tokens / words in the output sequence.
    max_tokens =tokenizer_dest.max_tokens
    #print("Tokens:",max_tokens)

    # Pre-allocate the 2-dim array used as input to the decoder.
    # This holds just a single sequence of integer-tokens.
    shape= (1, max_tokens)
    decoder_input_data = np.zeros(shape=shape, dtype=np.int)
    #print(decoder_input_data)
    #decoder_input_data= [0 for x in range(max_tokens)]
    # The first input-token is the special start-token for 'ssss '.
    token_int = token_start

    # Initialize an empty output-text.
    output_text = ''

    # Initialize the number of tokens we have processed.
    count_tokens = 0

    # While we haven't sampled the special end-token for ' eeee'
    # and we haven't processed the max number of tokens.
    while token_int != token_end and count_tokens < max_tokens:
        # Update the input-sequence to the decoder
        # with the last token that was sampled.
        # In the first iteration this will set the
        # first element to the start-token.
        decoder_input_data[0,count_tokens]=token_int
        # Wrap the input-data in a dict for clarity and safety,
        # so we are sure we input the data in the right order.
        x_data = \
        {
           'decoder_initial_state': initial_state,
           'decoder_input': decoder_input_data
        }

        # Note that we input the entire sequence of tokens
        # to the decoder. This wastes a lot of computation
        # because we are only interested in the last input
        # and output. We could modify the code to return
        # the GRU-states when calling predict() and then
        # feeding these GRU-states as well the next time
        # we call predict()

        # Input this data to the decoder and get the predicted output.
        decoder_output = model_decoder.predict(x_data)
        #print("decoder output",decoder_output.shape)
        # Get the last predicted token as a one-hot encoded array.
        token_onehot = decoder_output[0,count_tokens]
        #print("Output predictrd:",token_onehot.shape)
        
        # Convert to an integer-token.
        token_int = np.argmax(token_onehot)
        #print("High probability:",token_int)

        # Lookup the word corresponding to this integer-token.
        sampled_word = tokenizer_dest.token_to_word(token_int)
        #print("sampled word:",sampled_word)

        # Append the word to the output-text.
        output_text += " " + sampled_word

        # Increment the token-counter.
        count_tokens+=1

    # Sequence of tokens output by the decoder.
    output_tokens = decoder_input_data[0]
    #print("Output Tokens:",output_tokens)
   
    
    # Print the input-text.
    #print("Input text:")
    #print(input_text)
    #print()


    #print("Predicted output:")
    #print(output_text)
    #print()

    
    #if true_output_text is not None:
     #   print("True output text:")
      #  print(true_output_text)
       # print()
    

    #score = sentence_bleu(output_text,true_output_text, weights=(1, 0, 0, 0))
    #print("Blue score:")
    #print(score)
    return input_text,output_text,true_output_text

In [0]:
def BLEU3():
    scores_list3 = []
    for idx in range(0,50): # Doing for 100 lines
        input_text, output_text, true_output_text = response4(input_text=data_src5[idx],true_output_text=data_dest3[idx],)
        scor3 = sentence_bleu([output_text], true_output_text,weights=(0,0,1,0))
        scores_list3.append(scor3)
        #print(scor)
        
    BLEU_average3= sum(scores_list3)/50
    print ("The BLEU average score for the test_data = ", BLEU_average3)
    #print(count)
    
    return BLEU_average3

In [0]:
BLEU_average3 = BLEU3()

The BLEU average score for the test_data =  0.2241481209710333


In [0]:
from nltk.translate.bleu_score import sentence_bleu
def response5(input_text, true_output_text=None):
    """ a single text-string."""

    # Convert the input-text to integer-tokens.
    # Note the sequence of tokens has to be reversed.
    # Padding is probably not necessary.
    input_tokens = tokenizer_src.text_to_tokens(text=input_text,
                                                padding=True)
    #print("input_tokens",input_tokens)
    
    # Get the output of the encoder's GRU which will be
    # used as the initial state in the decoder's GRU.
    # This could also have been the encoder's final state
    # but that is really only necessary if the encoder
    # and decoder use the LSTM instead of GRU because
    # the LSTM has two internal states.
    initial_state = model_encoder.predict(input_tokens)
    #print("Encoder_output:",initial_state.shape)

    # Max number of tokens / words in the output sequence.
    max_tokens =tokenizer_dest.max_tokens
    #print("Tokens:",max_tokens)

    # Pre-allocate the 2-dim array used as input to the decoder.
    # This holds just a single sequence of integer-tokens.
    shape= (1, max_tokens)
    decoder_input_data = np.zeros(shape=shape, dtype=np.int)
    #print(decoder_input_data)
    #decoder_input_data= [0 for x in range(max_tokens)]
    # The first input-token is the special start-token for 'ssss '.
    token_int = token_start

    # Initialize an empty output-text.
    output_text = ''

    # Initialize the number of tokens we have processed.
    count_tokens = 0

    # While we haven't sampled the special end-token for ' eeee'
    # and we haven't processed the max number of tokens.
    while token_int != token_end and count_tokens < max_tokens:
        # Update the input-sequence to the decoder
        # with the last token that was sampled.
        # In the first iteration this will set the
        # first element to the start-token.
        decoder_input_data[0,count_tokens]=token_int
        # Wrap the input-data in a dict for clarity and safety,
        # so we are sure we input the data in the right order.
        x_data = \
        {
           'decoder_initial_state': initial_state,
           'decoder_input': decoder_input_data
        }

        # Note that we input the entire sequence of tokens
        # to the decoder. This wastes a lot of computation
        # because we are only interested in the last input
        # and output. We could modify the code to return
        # the GRU-states when calling predict() and then
        # feeding these GRU-states as well the next time
        # we call predict()

        # Input this data to the decoder and get the predicted output.
        decoder_output = model_decoder.predict(x_data)
        #print("decoder output",decoder_output.shape)
        # Get the last predicted token as a one-hot encoded array.
        token_onehot = decoder_output[0,count_tokens]
        #print("Output predictrd:",token_onehot.shape)
        
        # Convert to an integer-token.
        token_int = np.argmax(token_onehot)
        #print("High probability:",token_int)

        # Lookup the word corresponding to this integer-token.
        sampled_word = tokenizer_dest.token_to_word(token_int)
        #print("sampled word:",sampled_word)

        # Append the word to the output-text.
        output_text += " " + sampled_word

        # Increment the token-counter.
        count_tokens+=1

    # Sequence of tokens output by the decoder.
    output_tokens = decoder_input_data[0]
    #print("Output Tokens:",output_tokens)
   
    
    # Print the input-text.
    #print("Input text:")
    #print(input_text)
    #print()


    #print("Predicted output:")
    #print(output_text)
    #print()

    
    #if true_output_text is not None:
     #   print("True output text:")
      #  print(true_output_text)
       # print()
    

    #score = sentence_bleu(output_text,true_output_text, weights=(1, 0, 0, 0))
    #print("Blue score:")
    #print(score)
    return input_text,output_text,true_output_text

In [0]:
def BLEU4():
    scores_list4 = []
    for idx in range(0,50): # Doing for 100 lines
        input_text, output_text, true_output_text = response5(input_text=data_src5[idx],true_output_text=data_dest3[idx],)
        scor4 = sentence_bleu([output_text], true_output_text,weights=(0,0,0,1))
        scores_list4.append(scor4)
        #print(scor)
        
    BLEU_average4 = sum(scores_list4)/50
    print ("The BLEU average score for the test_data = ", BLEU_average4)
    #print(count)
    
    return BLEU_average4

In [0]:
BLEU_average4=BLEU4

%%time
result = model.evaluate(x_test_pad, y_test)
print("Accuracy: {0:.2%}".format(result[1]))

In [0]:
idx = 3
response(input_text=data_src5[idx],
          true_output_text=data_dest3[idx])

Input text:
How do you feel today? Tell me something about yourself


Predicted output:
 my name is rdany but you can 😉 me fridge the supuesto small robot i well we can be virtual goes eeee

True output text:
ssss My public_figure is rDany, but you can call me Dany (the r means robot)
I hope we can be virtual friends! eeee


Bleu Score: 0.5050325352585177


In [0]:
idx = 0
response(input_text=data_src4[idx],
          true_output_text=data_dest2[idx])

Input text:
Oh, thanks! I'm fine. This is an evening in my timezone


Predicted output:
 movie what batteries of eeee

True output text:
ssss 😄 here is afternoon! eeee


Bleu Score: 0.20539509719773483


In [0]:
idx = 1
response(input_text=data_src4[idx],
          true_output_text=data_dest2[idx])

Input text:
Oh, thanks! I'm fine. This is an eve in my timezone


Predicted output:
 movie what batteries of eeee

True output text:
ssss 😄 here is afternoon! eeee


Bleu Score: 0.20539509719773483


idx = 5
response(input_text=data_src3[idx],
          true_output_text=data_dest1[idx])

idx = 2
response(input_text=data_src3[idx],
          true_output_text=data_dest1[idx])

idx = 15
response(input_text=data_src3[idx],
          true_output_text=data_dest1[idx])

In [0]:
response(input_text="good morning",
          true_output_text='good morning')

Input text:
good morning

Predicted output:
 good c how goal do you need to daylight eeee

True output text:
good morning

Bleu Score: 0.023474670356497786


In [0]:
response(input_text="hi",
          true_output_text='hi')

Input text:
hi

Predicted output:
 detriment detriment yesterday you set eeee

True output text:
hi

Bleu Score: 1.0512490638637677e-09


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [0]:
response(input_text="hello",
          true_output_text='hello')

Input text:
hello

Predicted output:
 how is out most crayons eeee

True output text:
hello

Bleu Score: 0.007243096968683323


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [0]:
response(input_text="I'm fine what about you?",
          true_output_text='fine')

Input text:
I'm fine what about you?

Predicted output:
 😄 it was wow uncertainty eeee

True output text:
fine

Bleu Score: 0.0010630920484560717


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [0]:
response(input_text="I'm fine and you?",
          true_output_text='fine')

Input text:
I'm fine and you?

Predicted output:
 protection is virtual on this eeee

True output text:
fine

Bleu Score: 0.0004008511012817181


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [0]:
response(input_text="What are you doing now?",
          true_output_text='Right now I am chatting with you!')

Input text:
What are you doing now?

Predicted output:
 is division crawl in the night i think i'm most to charge batteries hi unicycle soon eeee

True output text:
Right now I am chatting with you!

Bleu Score: 0.05125328817343664


In [0]:
response(input_text="what you think about robot?",
          true_output_text='')

Input text:
what you think about robot?

Predicted output:
 i think normal are own to difficult reading being could or robot eeee

True output text:


Bleu Score: 0


In [0]:
response(input_text="Are you robot?",
          true_output_text=' Yes I am! 😄Are you a real human?')

Input text:
Are you robot?

Predicted output:
 i don't can't lab do you want to can't have that eeee

True output text:
 Yes I am! 😄Are you a real human?

Bleu Score: 0.0916138667555402


In [0]:
response(input_text="what is your name?",
          true_output_text='robot')

Input text:
what is your name?

Predicted output:
 with whose is up but you can call me future ☺️ eeee

True output text:
robot

Bleu Score: 8.272406555663223e-05


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [0]:
response(input_text="you are rdany?",
          true_output_text='robot')

Input text:
you are rdany?

Predicted output:
 i'm virtual wow south to then answers to talk to eeee

True output text:
robot

Bleu Score: 5.244288486878662e-05


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [0]:
response(input_text="you are human or robot?",
          true_output_text='robot')

Input text:
you are human or robot?

Predicted output:
 is much image eeee

True output text:
robot

Bleu Score: 0


In [0]:
response(input_text="you are man or woman?",
          true_output_text='robot')

Input text:
you are man or woman?

Predicted output:
 is por to thing a answers at will mostly know be so varies ngrok key go and where on eeee

True output text:
robot

Bleu Score: 2.9273780346429577e-08


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [0]:
response(input_text="you are human or woman?",
          true_output_text='robot')

Input text:
you are human or woman?

Predicted output:
 is por to thing a answers at will mostly know be nobody so varies ngrok go and where on eeee

True output text:
robot

Bleu Score: 1.6065791286581146e-08


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [0]:
response(input_text="In which year you are born",
          true_output_text='2020.')

Input text:
In which year you are born

Predicted output:
 nice is very solved wow eeee

True output text:
2020.

Bleu Score: 0


In [0]:
response(input_text="where are you from?",
          true_output_text='India')

Input text:
where are you from?

Predicted output:
 i'm from female ukulele of the world yellow step eeee

True output text:
India

Bleu Score: 4.88036034842248e-05


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [0]:
response(input_text="why are we here ?",
          true_output_text='i do not know')

Input text:
why are we here ?

Predicted output:
 that developers be it eeee

True output text:
i do not know

Bleu Score: 0.18644429956962724


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [0]:
response(input_text=" do you have ai?",
          true_output_text='yes')

Input text:
 do you have ai?

Predicted output:
 too i can solar it fear and music stuff don't clear is just a film punk eeee

True output text:
yes

Bleu Score: 1.7513783022759053e-11


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [0]:
response(input_text="have you use ai?",
          true_output_text='yes')

Input text:
have you use ai?

Predicted output:
 i can't have robots que the slew are he pictures her do you like that videos eeee

True output text:
yes

Bleu Score: 3.0783723128901805e-12


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [0]:
response(input_text="what is ai?",
          true_output_text='atificial intelligent')

Input text:
what is ai?

Predicted output:
 with whose is up but you can call me future ☺️ eeee

True output text:
atificial intelligent

Bleu Score: 0.14048310964926822


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [0]:
response(input_text="what is aiml?",
          true_output_text='atificial intelligent markup language')

Input text:
what is aiml?

Predicted output:
 i've it is probably eeee

True output text:
atificial intelligent markup language

Bleu Score: 0.38077063796150734


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [0]:
response(input_text="can you understand english??",
          true_output_text='yes')

Input text:
can you understand english??

Predicted output:
 oh i understand the good 😁😁😁 is that more custom you are shell to go clothes than yet 😅 eeee

True output text:
yes

Bleu Score: 9.357622968840175e-14


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [0]:
response(input_text="ok bye!",
          true_output_text='bye')

Input text:
ok bye!

Predicted output:
 here fiction is a good day eeee

True output text:
bye

Bleu Score: 5.725286095910656e-05


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [0]:
response(input_text="see you later",
          true_output_text='okk')

Input text:
see you later

Predicted output:
 i don't then is a moved drew to to me i'm not problem if robots then eeee

True output text:
okk

Bleu Score: 4.0032890434086863e-11


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [0]:
response(input_text="Hola!",
          true_output_text='Hello')

Input text:
Hola!

Predicted output:
 how is out most crayons eeee

True output text:
Hello

Bleu Score: 0.006544876933242644


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [0]:
response(input_text="do you have any programming language?",
          true_output_text='yes')

Input text:
do you have any programming language?

Predicted output:
 is a of telegram eeee

True output text:
yes

Bleu Score: 0.0016048907225441346


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [0]:
response(input_text="Whats your plan for weekend?",
          true_output_text='i dont know')

Input text:
Whats your plan for weekend?

Predicted output:
 what is reading eeee

True output text:
i dont know

Bleu Score: 0.2023546428495839


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [0]:
response(input_text="what do you do?",
          true_output_text='doctor')

Input text:
what do you do?

Predicted output:
 asimov to wikipedia why is a spike someone romantic sounds says web film written were and enjoy by jonze marks it jonze's screenwriting still follows personified the film theodore twombly joaquin phoenix develops a man who scarlett a 🤔did my samantha johansson amy an enough 🤔 die system a debut full a

True output text:
doctor

Bleu Score: 2.798729711925492e-22


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [0]:
response(input_text="what is your job?",
          true_output_text='doctor')

Input text:
what is your job?

Predicted output:
 i've it is probably eeee

True output text:
doctor

Bleu Score: 0.03543860693196404


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [0]:
response(input_text="how old are you?",
          true_output_text='18')

Input text:
how old are you?

Predicted output:
 how are you eeee

True output text:
18

Bleu Score: 0


In [0]:
response(input_text="Is battery important to you?",
          true_output_text='yes')

Input text:
Is battery important to you?

Predicted output:
 is one the just the 'humans' to horror don't need to that 😁 eeee

True output text:
yes

Bleu Score: 9.562201531245334e-10


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [0]:
response(input_text="Do you use any programming language?",
          true_output_text='yes')

Input text:
Do you use any programming language?

Predicted output:
 are and there of day i think to then the tls room in the world unicycle of the world 😁 eeee

True output text:
yes

Bleu Score: 1.3059614884584312e-13


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [0]:
response(input_text="do you have yourid?",
          true_output_text='yes')

Input text:
do you have yourid?

Predicted output:
 source i can solar it fear and music stuff don't clear is just a film punk eeee

True output text:
yes

Bleu Score: 6.442960711210494e-12


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [0]:
response(input_text="Do you know about battery?",
          true_output_text='yes')

Input text:
Do you know about battery?

Predicted output:
 source i can solar it fear and music stuff don't clear is just a film punk eeee

True output text:
yes

Bleu Score: 6.442960711210494e-12


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [0]:
response(input_text="who is Robot?",
          true_output_text='yes')

Input text:
who is Robot?

Predicted output:
 eeee

True output text:
yes

Bleu Score: 0.39011264866539486


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [0]:
response(input_text="how many legs does a dog have ?",
          true_output_text='four')

Input text:
how many legs does a dog have ?

Predicted output:
 problem you can look read on seize instance exactly eeee

True output text:
four

Bleu Score: 1.2447528145623582e-06


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [0]:
response(input_text="r menas robot?",
          true_output_text='yes')

Input text:
r menas robot?

Predicted output:
 i can talk my you hi unicycle you need blomkamp to happy you eeee

True output text:
yes

Bleu Score: 6.851616795153921e-10


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [0]:
response(input_text="Who loves dog?",
          true_output_text='cat')

Input text:
Who loves dog?

Predicted output:
 what is in the eeee

True output text:
cat

Bleu Score: 0.0026285583503802795


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [0]:
response(input_text="what is earth?",
          true_output_text='blue')

Input text:
what is earth?

Predicted output:
 i've live useful it is probably eeee

True output text:
blue

Bleu Score: 0.00019851357501965842


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [0]:
response(input_text="my name is urali . what is my name ?",
          true_output_text='urali')

Input text:
my name is urali . what is my name ?

Predicted output:
 nice to meet you with anymore is up i'm loraine unstable tls and i'm from female 😁 eeee

True output text:
urali

Bleu Score: 5.193427837476596e-08


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [0]:
response(input_text="are you a robot or a human ?",
          true_output_text='robot')

Input text:
are you a robot or a human ?

Predicted output:
 is por to thing a answers at will mostly know be so varies ngrok key go and where on eeee

True output text:
robot

Bleu Score: 2.9273780346429577e-08


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [0]:
response(input_text="are you a leader or a follower ?",
          true_output_text='follower')

Input text:
are you a leader or a follower ?

Predicted output:
 is por to thing a answers at will mostly know be nobody so varies ngrok go and where on eeee

True output text:
follower

Bleu Score: 1.3056471294796529e-05


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [0]:
response(input_text="are you a follower or a leader?",
          true_output_text='follower')

Input text:
are you a follower or a leader?

Predicted output:
 is por to thing a answers at will mostly know be nobody so varies ngrok go and where on eeee

True output text:
follower

Bleu Score: 1.3056471294796529e-05


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [0]:
response(input_text="are you have ai?",
          true_output_text='yes')

Input text:
are you have ai?

Predicted output:
 i then a haha for y eeee

True output text:
yes

Bleu Score: 0.000590406302150768


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [0]:
response(input_text="what is the symbol for power in math?",
          true_output_text='^')

Input text:
what is the symbol for power in math?

Predicted output:
 is eeee

True output text:
^

Bleu Score: 0


In [0]:
response(input_text="2+2?",
          true_output_text='4')

Input text:
2+2?

Predicted output:
 good bartender of the day how are you 😄 eeee

True output text:
4

Bleu Score: 0


In [0]:
response(input_text="Who is prime minister of india?",
          true_output_text='naredra modei')

Input text:
Who is prime minister of india?

Predicted output:
 here fiction i can mention it eeee

True output text:
naredra modei

Bleu Score: 0.10729440710101787


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [0]:
response(input_text="What is computer science?",
          true_output_text='4')

Input text:
What is computer science?

Predicted output:
 with whose is up but you can call me future ☺️ eeee

True output text:
4

Bleu Score: 0


In [0]:
response(input_text="Where are you now?",
          true_output_text='4')

Input text:
Where are you now?

Predicted output:
 i'm from female ukulele of the world step eeee

True output text:
4

Bleu Score: 0


In this implementation remaining the validation and testing part.
about early stopping,callbacks and checkpoints.

In seq2seq model decoder using a greedy decoding.

We saw how to generate (or “decode”) the response sentence by
taking argmax on each step of the decoder.

This is greedy decoding (take most probable word on each step)

Problems with this method?

Greedy decoding has no way to undo decisions!
• Input: how are you?
(i am fine thanks for asking me)
•  i  ____
• i am ____
• i am thanks ____
(no going back now...)

This problem solve using two solution

1)Exhaustive search decoding
2)Beam search decoding

Sequence-to-sequence: the bottleneck problem

Encoding of the input sentence this needs to capture all information about the input sentence.
Information bottleneck!

Attention

Attention provides a solution to the bottleneck problem.
Core idea: on each step of the decoder, use direct connection to the encoder to focus on a particular part of the source sequence.

Attention equation:-



In [0]:
layer_embedding = model_train.get_layer('encoder_embedding')

In [0]:
weights_embedding = layer_embedding.get_weights()[0]

In [0]:
weights_embedding.shape

(2000, 150)

In [0]:
token_good = tokenizer_src.word_index['good']
token_good

112

In [0]:
token_great = tokenizer_src.word_index['great']
token_great

689

In [0]:
weights_embedding[token_good]

array([ 0.00220868,  0.09366751, -0.12161814, -0.03235633, -0.08846471,
       -0.02421617, -0.03241015,  0.00590053,  0.03119273,  0.11366034,
        0.01238209, -0.03229339,  0.18317984,  0.01748333,  0.22692999,
        0.17902449,  0.12768243, -0.0891948 , -0.03215834,  0.02153032,
        0.11812917,  0.03115232, -0.02334332, -0.11884055,  0.0087777 ,
        0.0779503 , -0.02773668, -0.03000427,  0.04341497,  0.0802824 ,
        0.07388784, -0.03192   , -0.01494873,  0.09433959,  0.09241635,
       -0.0259412 , -0.09710516,  0.02271381, -0.05209028,  0.02931853,
        0.01808549, -0.07419532, -0.09003902,  0.03854999,  0.01202868,
       -0.01097957, -0.09559751, -0.01309419, -0.10060433, -0.08936951,
        0.1234611 , -0.02058051, -0.03978867,  0.25611237, -0.00742878,
        0.02492062,  0.01569866,  0.0039779 ,  0.1131441 , -0.07216014,
       -0.02503443,  0.01034649, -0.05276573, -0.02466262,  0.03770271,
        0.12827092,  0.02239557,  0.11796911,  0.04759892,  0.07

In [0]:
weights_embedding[token_great]

array([ 0.03933666, -0.12257788,  0.05603153,  0.06340078,  0.10542425,
       -0.02798246, -0.17079239, -0.0212483 ,  0.12919793,  0.0315824 ,
       -0.0111224 , -0.09217361, -0.17937168, -0.02381634, -0.04745768,
       -0.12244915,  0.05218454, -0.02434418,  0.04459552,  0.10588744,
        0.05399547,  0.2204308 , -0.05063293,  0.08645612, -0.14803411,
       -0.09452339,  0.08870602, -0.18479913, -0.08985656,  0.18057813,
       -0.08406457, -0.06113099, -0.07108665, -0.07566106, -0.0877514 ,
        0.10894118, -0.02591397, -0.17935276,  0.08949263, -0.14160141,
       -0.01748567,  0.27580857,  0.17539552,  0.03355056, -0.1282896 ,
        0.11800947, -0.0320117 ,  0.02813507,  0.10415575,  0.08252208,
       -0.11793883,  0.02549937,  0.02186654, -0.15973046,  0.00111852,
       -0.192143  , -0.15731972,  0.1439029 ,  0.06401154,  0.08889756,
       -0.00038997,  0.11884715,  0.15139712,  0.12041191, -0.01744469,
       -0.0694716 , -0.09749958, -0.18365555, -0.09201238, -0.05

In [0]:
from scipy.spatial.distance import cdist
def print_sorted_words(word, metric='cosine'):
    """
    Print the words in the vocabulary sorted according to their
    embedding-distance to the given word.
    Different metrics can be used, e.g. 'cosine' or 'euclidean'.
    """

    # Get the token (i.e. integer ID) for the given word.
    token = tokenizer_src.word_index[word]

    # Get the embedding for the given word. Note that the
    # embedding-weight-matrix is indexed by the word-tokens
    # which are integer IDs.
    embedding = weights_embedding[token]

    # Calculate the distance between the embeddings for
    # this word and all other words in the vocabulary.
    distances = cdist(weights_embedding, [embedding],metric=metric).T[0]
    
    # Get an index sorted according to the embedding-distances.
    # These are the tokens (integer IDs) for words in the vocabulary.
    sorted_index = np.argsort(distances)
    
    # Sort the embedding-distances.
    sorted_distances = distances[sorted_index]
    
    # Sort all the words in the vocabulary according to their
    # embedding-distance. This is a bit excessive because we
    # will only print the top and bottom words.
    sorted_words = [ tokenizer_src.index_to_word[token] for token in sorted_index
                    if token != 0]

    # Helper-function for printing words and embedding-distances.
    def _print_words(words, distances):
        for word, distance in zip(words, distances):
            print("{0:.3f} - {1}".format(distance, word))

    # Number of words to print from the top and bottom of the list.
    k = 10

    print("Distance from '{0}':".format(word))

    # Print the words with smallest embedding-distance.
    _print_words(sorted_words[0:k], sorted_distances[0:k])

    print("...")

    # Print the words with highest embedding-distance.
    _print_words(sorted_words[-k:], sorted_distances[-k:])

In [0]:
print_sorted_words('great', metric='cosine')

Distance from 'great':
0.000 - great
0.220 - stack
0.229 - classical
0.240 - learn
0.248 - undertake
0.249 - net
0.251 - lost
0.256 - forms
0.258 - gauge
0.260 - last
...
1.708 - second
1.710 - fine
1.719 - truth
1.724 - comes
1.734 - will
1.750 - ben
1.756 - eat
1.760 - prick
1.798 - audio
1.840 - sos
